


### **บทนำ**

**วัตถุประสงค์หลัก:** เพื่อฝึกฝนทักษะสำคัญ 2 ประการที่ได้เรียนรู้จาก Notebooks file = `01_ Structured Data to Insightful.ipynb` และ `03_Structured Outputs_with_Pydatic.ipynb` ได้แก่:

1.  **การสกัดข้อมูล (Structured Data Extraction):** 📝 การแปลงข้อความที่ไม่มีโครงสร้าง (เช่น รีวิว, อีเมล, แผนการเดินทาง) ให้กลายเป็นข้อมูลที่มีโครงสร้างและเป็นระเบียบ โดยใช้ Pydantic Model เป็นตัวกำหนดรูปแบบ
2.  **การสร้างเนื้อหา (Insightful Content Generation):** 🤖 การนำข้อมูลที่มีโครงสร้างที่สกัดมาได้ มาใช้เป็น "วัตถุดิบ" ในการสร้างสรรค์เนื้อหาใหม่ๆ ที่มีความหมายและนำไปใช้ประโยชน์ต่อได้ เช่น การเขียนโพสต์ตอบกลับ, การร่างอีเมล, หรือการจัดทำตารางสรุป

-----

### **โจทย์ข้อที่ 1: ตอบกลับรีวิวร้านกาแฟ ☕**

**สถานการณ์:** คุณเป็นเจ้าของร้านกาแฟที่ใส่ใจลูกค้า และต้องการใช้ AI ช่วยอ่านรีวิวที่มีหลากหลายรูปแบบ จากนั้นนำข้อมูลที่ได้มาสร้างคำขอบคุณที่เป็นส่วนตัวและน่าประทับใจ เพื่อโพสต์ลงโซเชียลมีเดีย

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลจากรีวิว**

1.  **กำหนดโครงสร้างด้วย Pydantic:** สร้าง Pydantic Model เพื่อบังคับให้ AI สกัดข้อมูลออกมาในรูปแบบที่ต้องการ ดังนี้:
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class CafeReview(BaseModel):
        cafe_name: str = Field(description="ชื่อของร้านกาแฟที่ถูกรีวิว")
        items_ordered: List[str] = Field(description="ลิสต์ของเมนูที่ลูกค้ารีวิวว่าได้สั่งไป")
        rating: int = Field(description="คะแนนดาวที่ลูกค้าให้ เป็นตัวเลข 1-5")
        comment: str = Field(description="ความคิดเห็นโดยรวมของลูกค้า")
    ```
2.  **เรียกใช้ LLM:** เขียนโค้ดเพื่อส่งข้อความรีวิว (จากชุดข้อมูลทดสอบด้านล่าง) พร้อมกับ `CafeReview` model เข้าไปใน LLM เพื่อให้ AI สกัดข้อมูลออกมาเป็น object ตามโครงสร้างที่กำหนด

**Part 2: สร้างโพสต์ขอบคุณ**

1.  **สร้าง Prompt:** นำ object ที่ได้จาก Part 1 มาสร้างเป็น Prompt ที่มีรายละเอียด เพื่อสั่งให้ AI เขียนแคปชั่น
2.  **เรียกใช้ LLM:** สั่งให้ AI สร้างข้อความตอบกลับที่ดูเป็นธรรมชาติและมีการอ้างอิงถึงเมนูที่ลูกค้าสั่งและคะแนนที่ให้

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

ใช้ลิสต์ต่อไปนี้เพื่อทดสอบว่าโค้ดของคุณสามารถจัดการกับรีวิวที่หลากหลายได้หรือไม่

```python
queries_cafe_reviews = [
    # Query 1: รีวิวมาตรฐาน ตรงไปตรงมา
    "ไปลองร้าน 'The Reading Room' มาเมื่อวาน สั่งลาเต้ร้อนกับบราวนี่ไปหนึ่งชิ้น กาแฟรสชาติดีมาก แต่บราวนี่เนื้อแน่นไปหน่อย โดยรวมให้ 4 ดาวครับ",

    # Query 2: ใช้ภาษาไม่เป็นทางการ มีการให้คะแนนแบบเศษส่วน
    "เพิ่งกลับมาจากร้าน Bake & Brew คือดีย์มากกก สั่งชาเขียวมัทฉะกับครัวซองต์ไข่เค็มไปคือที่สุด! แต่ร้านเล็กไปนิดนึง คนเยอะเลยต้องรอคิว ให้ 3/5 ดาวละกัน",

    # Query 3: รีวิวเชิงลบ และไม่ได้สั่งเครื่องดื่ม
    "ประสบการณ์ที่ร้าน 'Sip & Smile' ถือว่าน่าผิดหวัง สั่งแค่เค้กมะพร้าวมาชิ้นเดียว แต่รอไป 20 นาที แถมรสชาติก็ธรรมดามาก ไม่ประทับใจเลย ให้ 1 ดาวพอ",

    # Query 4: ไม่ได้บอกชื่อร้านตรงๆ และให้คะแนนแบบบรรยาย
    "วันนี้ไปคาเฟ่เปิดใหม่แถวอารีย์ที่ชื่อ The Hidden Cup มาค่ะ สั่งอเมริกาโน่เย็นกับครัวซองต์แฮมชีส อร่อยทั้งสองอย่างเลยค่ะ ประทับใจมาก เอาไปเลย 5 ดาวเต็ม!",

    # Query 5: ประโยคซับซ้อน มีทั้งคำชมและคำติปนกัน
    "แม้ว่าบรรยากาศที่ร้าน 'Lo-fi Coffee' จะตกแต่งได้สวยงาม และกาแฟ Dirty ที่สั่งไปจะทำออกมาได้ดี แต่ก็ต้องหักคะแนนเรื่องที่จอดรถที่ไม่มีเลย และชีสเค้กหน้าไหม้ที่รสชาติหวานเกินไปมาก สรุปแล้วเลยขอให้แค่ 2 ดาว"
]
```



In [1]:
queries_cafe_reviews = [
    # Query 1: รีวิวมาตรฐาน ตรงไปตรงมา
    "ไปลองร้าน 'The Reading Room' มาเมื่อวาน สั่งลาเต้ร้อนกับบราวนี่ไปหนึ่งชิ้น กาแฟรสชาติดีมาก แต่บราวนี่เนื้อแน่นไปหน่อย โดยรวมให้ 4 ดาวครับ",

    # Query 2: ใช้ภาษาไม่เป็นทางการ มีการให้คะแนนแบบเศษส่วน
    "เพิ่งกลับมาจากร้าน Bake & Brew คือดีย์มากกก สั่งชาเขียวมัทฉะกับครัวซองต์ไข่เค็มไปคือที่สุด! แต่ร้านเล็กไปนิดนึง คนเยอะเลยต้องรอคิว ให้ 3/5 ดาวละกัน",

    # Query 3: รีวิวเชิงลบ และไม่ได้สั่งเครื่องดื่ม
    "ประสบการณ์ที่ร้าน 'Sip & Smile' ถือว่าน่าผิดหวัง สั่งแค่เค้กมะพร้าวมาชิ้นเดียว แต่รอไป 20 นาที แถมรสชาติก็ธรรมดามาก ไม่ประทับใจเลย ให้ 1 ดาวพอ",

    # Query 4: ไม่ได้บอกชื่อร้านตรงๆ และให้คะแนนแบบบรรยาย
    "วันนี้ไปคาเฟ่เปิดใหม่แถวอารีย์ที่ชื่อ The Hidden Cup มาค่ะ สั่งอเมริกาโน่เย็นกับครัวซองต์แฮมชีส อร่อยทั้งสองอย่างเลยค่ะ ประทับใจมาก เอาไปเลย 5 ดาวเต็ม!",

    # Query 5: ประโยคซับซ้อน มีทั้งคำชมและคำติปนกัน
    "แม้ว่าบรรยากาศที่ร้าน 'Lo-fi Coffee' จะตกแต่งได้สวยงาม และกาแฟ Dirty ที่สั่งไปจะทำออกมาได้ดี แต่ก็ต้องหักคะแนนเรื่องที่จอดรถที่ไม่มีเลย และชีสเค้กหน้าไหม้ที่รสชาติหวานเกินไปมาก สรุปแล้วเลยขอให้แค่ 2 ดาว"
]

queries_cafe_reviews

["ไปลองร้าน 'The Reading Room' มาเมื่อวาน สั่งลาเต้ร้อนกับบราวนี่ไปหนึ่งชิ้น กาแฟรสชาติดีมาก แต่บราวนี่เนื้อแน่นไปหน่อย โดยรวมให้ 4 ดาวครับ",
 'เพิ่งกลับมาจากร้าน Bake & Brew คือดีย์มากกก สั่งชาเขียวมัทฉะกับครัวซองต์ไข่เค็มไปคือที่สุด! แต่ร้านเล็กไปนิดนึง คนเยอะเลยต้องรอคิว ให้ 3/5 ดาวละกัน',
 "ประสบการณ์ที่ร้าน 'Sip & Smile' ถือว่าน่าผิดหวัง สั่งแค่เค้กมะพร้าวมาชิ้นเดียว แต่รอไป 20 นาที แถมรสชาติก็ธรรมดามาก ไม่ประทับใจเลย ให้ 1 ดาวพอ",
 'วันนี้ไปคาเฟ่เปิดใหม่แถวอารีย์ที่ชื่อ The Hidden Cup มาค่ะ สั่งอเมริกาโน่เย็นกับครัวซองต์แฮมชีส อร่อยทั้งสองอย่างเลยค่ะ ประทับใจมาก เอาไปเลย 5 ดาวเต็ม!',
 "แม้ว่าบรรยากาศที่ร้าน 'Lo-fi Coffee' จะตกแต่งได้สวยงาม และกาแฟ Dirty ที่สั่งไปจะทำออกมาได้ดี แต่ก็ต้องหักคะแนนเรื่องที่จอดรถที่ไม่มีเลย และชีสเค้กหน้าไหม้ที่รสชาติหวานเกินไปมาก สรุปแล้วเลยขอให้แค่ 2 ดาว"]



### **โจทย์ข้อที่ 2: สรุปรายการสั่งซื้อของลูกค้า 🛍️**

**สถานการณ์:** ลูกค้าส่งอีเมลสั่งซื้อของหลายรายการในรูปแบบที่แตกต่างกัน คุณต้องใช้ AI ช่วยแปลงอีเมลเหล่านั้นให้เป็นข้อมูลที่มีโครงสร้าง เพื่อให้ง่ายต่อการเตรียมจัดของและส่งอีเมลยืนยันกลับไปหาลูกค้า

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลรายการสั่งซื้อ**

1.  **กำหนดโครงสร้างแบบซ้อนกัน:** สร้าง Pydantic Model 2 ชั้น เพื่อจัดการกับข้อมูลที่มีความสัมพันธ์แบบหนึ่งต่อกลุ่ม (One-to-Many)
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class OrderItem(BaseModel):
        item_name: str = Field(description="ชื่อสินค้าที่สั่ง")
        quantity: int = Field(description="จำนวนสินค้าชิ้นนั้นๆ")

    class Order(BaseModel):
        customer_name: str = Field(description="ชื่อของลูกค้าผู้สั่งซื้อ")
        items: List[OrderItem] = Field(description="ลิสต์ของสินค้าทั้งหมดในออเดอร์")
    ```
2.  **เรียกใช้ LLM:** ส่งเนื้อหาอีเมล (จากชุดข้อมูลทดสอบ) พร้อมกับ `Order` model เข้าไปเพื่อให้ AI แปลงข้อมูลเป็น object ที่ถูกต้อง

**Part 2: สร้างอีเมลยืนยัน**

1.  **เตรียมข้อมูลสำหรับ Prompt:** เขียนโค้ดเพื่อวนลูป `items` ที่สกัดมาได้ เพื่อสร้างเป็นรายการสินค้า (string) และคำนวณจำนวนสินค้ารวม
2.  **สร้าง Prompt และเรียกใช้ LLM:** สั่งให้ AI ร่างอีเมลยืนยันการสั่งซื้อ โดยใช้ชื่อลูกค้า, รายการสินค้า, และจำนวนรวมที่เตรียมไว้

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

```python
queries_customer_orders = [
    # Query 1: รูปแบบมาตรฐาน ใช้ขีดนำหน้า
    "สวัสดีครับ ผมมานะ ต้องการสั่งของตามนี้ครับ:\n- หูฟังไร้สาย XYZ จำนวน 1 อัน\n- เคสกันกระแทก สีดำ 2 ชิ้น\n- สายชาร์จ USB-C ยาว 2 เมตร 1 เส้น",

    # Query 2: บอกจำนวนในประโยค และชื่ออยู่ท้ายอีเมล
    "รบกวนสั่งของหน่อยนะคะ อยากได้เมาส์ไร้สายสีชมพูหนึ่งอัน แล้วก็แผ่นรองเมาส์ลายแมวอีกสองแผ่นค่ะ ขอบคุณค่ะ - จากคุณวันดี",

    # Query 3: มีสินค้าที่ไม่ระบุจำนวน (ให้ตีความเป็น 1) และมีคำถามปนมา
    "สนใจสั่งคีย์บอร์ด Mechanical รุ่น K-1200 และขอพวงกุญแจลายอวกาศด้วยค่ะ ไม่แน่ใจว่าต้องสั่งขั้นต่ำเท่าไหร่คะ? ถ้าไม่ต้อง รบกวนจัดส่งได้เลย ชื่อผู้รับคือสมใจค่ะ",

    # Query 4: ระบุจำนวนด้วยตัวเลขในวงเล็บและคั่นด้วยจุลภาค
    "ผมปีเตอร์นะครับ ขอสั่ง, ครีมกันแดด (2), โฟมล้างหน้า (1), และสเปรย์น้ำแร่ (3) ครับ",

    # Query 5: รายการสั่งซื้อที่ซับซ้อนและมีการแก้ไข
    "ถึงแอดมิน ผมวิรัชครับ ขอสั่งเสื้อโปโลสีขาว 2 ตัวครับ ขนาด L นะครับ แล้วก็ขอกางเกงขาสั้นสีเทา 1 ตัว ขนาด M อ้อ! เกือบลืม เพิ่มถุงเท้าสีขาว 3 คู่ด้วยครับ"
]
```





### **โจทย์ข้อที่ 3: จัดตารางเที่ยว 🗺️**

**สถานการณ์:** เพื่อนส่งแผนการเดินทางมาให้ในรูปแบบข้อความบรรยาย ซึ่งอาจจะอ่านยากและสับสน คุณต้องใช้ AI ช่วยสกัดข้อมูลเวลาและกิจกรรมตามลำดับ แล้วจัดทำเป็นตารางแผนเที่ยวที่ชัดเจนและอ่านง่าย

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลแผนการเดินทาง**

1.  **กำหนดโครงสร้างสำหรับข้อมูลที่มีลำดับ:** สร้าง Pydantic Model เพื่อเก็บข้อมูลของแต่ละกิจกรรมตามลำดับเวลา
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class ScheduleItem(BaseModel):
        time: str = Field(description="เวลาของกิจกรรม")
        activity: str = Field(description="รายละเอียดของกิจกรรมที่จะทำ")

    class Itinerary(BaseModel):
        plan: List[ScheduleItem] = Field(description="ลิสต์ของแผนการเดินทางที่เรียงตามลำดับเวลา")
    ```
2.  **เรียกใช้ LLM:** ส่งข้อความแผนเที่ยว (จากชุดข้อมูลทดสอบ) พร้อม `Itinerary` model เข้าไป เพื่อให้ AI สกัดและเรียงลำดับกิจกรรมให้ถูกต้อง

**Part 2: สร้างตารางแผนเที่ยว**

1.  **สร้าง Prompt สำหรับการจัดรูปแบบ:** สั่งให้ LLM นำข้อมูล `plan` ที่เป็นลิสต์ของ object มาแปลงให้อยู่ในรูปแบบตาราง Markdown
2.  **เรียกใช้ LLM:** รับผลลัพธ์เป็น String ที่เป็นโค้ด Markdown สำหรับสร้างตาราง

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

```python
queries_travel_plans = [
    # Query 1: รูปแบบมาตรฐาน มีเวลาชัดเจน
    "ทริปหัวหินวันเดียว: 10:00 น. ไปถึงสถานีรถไฟหัวหิน ถ่ายรูปเล่น, 12:30 น. กินข้าวเที่ยงที่ร้านเจ๊เขียวซีฟู้ด, 14:00 น. เดินทางไปตลาดซิเคด้า, 18:00 น. เดินทางกลับ",

    # Query 2: ใช้การบอกลำดับและช่วงเวลา (เช้า, บ่าย, เย็น)
    "แผนเที่ยวเชียงรายของเรานะ เริ่มตอนเช้าไปวัดร่องขุ่นก่อนเลย หลังจากนั้นค่อยไปหาข้าวกินตอนกลางวัน บ่ายๆ ว่าจะแวะไปไร่ชาฉุยฟง แล้วปิดท้ายตอนเย็นที่สิงห์ปาร์ค",

    # Query 3: เล่าแบบไม่เรียงตามลำดับเวลา
    "โอเค สรุปแพลนน่านนะ ตอนเย็นเราจะไปเดินถนนคนเดินกัน แต่ว่าตอนบ่ายโมงเราต้องไปไหว้พระธาตุแช่แห้งก่อนนะ ส่วนตอนเช้าสุดเลย พอไปถึงเราจะไปวัดภูมินทร์กันก่อนเลย ประมาณ 9 โมง",

    # Query 4: มีกิจกรรมย่อยซ้อนอยู่ในกิจกรรมหลัก
    "แพลนสำหรับวันพรุ่งนี้ เริ่ม 9 โมงเช้า เจอกันที่ท่าเรือเพื่อไปเกาะล้าน พอไปถึงแล้ว กิจกรรมแรกคือดำน้ำดูปะการัง จากนั้นประมาณบ่ายโมงค่อยกินข้าวเที่ยงกันที่หาดตาแหวน แล้วค่อยเดินทางกลับตอนสี่โมงเย็น",

    # Query 5: ใช้ภาษาพูดและไม่มีเวลาที่ชัดเจนมากนัก
    "สรุปนะเพื่อนๆ เราไปถึงเขาใหญ่กันก่อนเที่ยง หาข้าวเที่ยงกินกันแถวนั้นแหละ พอกินเสร็จก็เข้าที่พักเก็บของ บ่ายแก่ๆ ค่อยออกไปเที่ยวคาเฟ่สวยๆ ซักที่นึง ตกเย็นค่อยไปกินหมูกระทะกัน"
]
```